In [1]:
from splender.image2 import SplenderImage
import jax.random as random
from jax import vmap
from interpax import Interpolator1D
from functools import partial

import numpy as np
import jax
import jax.numpy as jnp
from minimum_spanning_tree_inverse_spline_render import get_splines_from_frame

import matplotlib.pyplot as plt

import equinox as eqx

import jax.tree_util as jtu

import optax


In [2]:
dat = np.loadtxt('mnist_train.csv', delimiter=',')

In [3]:
def fit_spline(knots):
    x, y = knots[..., 0], knots[..., 1]
    
    s = jnp.linspace(0, 1, len(x))
    x_spline = Interpolator1D(s, x, method="cubic2")
    y_spline = Interpolator1D(s, y, method="cubic2")
    return x_spline, y_spline

def get_uniform_points(x_spline, y_spline, n_points_per_spline = 8):
    eps = 1e-6
    # Compute arc length
    dx_ds = vmap(partial(x_spline, dx=1))
    dy_ds = vmap(partial(y_spline, dx=1))
    s_fine = jnp.linspace(0, 1, n_points_per_spline)
    ds_vals = jnp.sqrt(dx_ds(s_fine)**2 + dy_ds(s_fine)**2 + eps)
    delta_s = s_fine[1] - s_fine[0]
    cumulative_length = jnp.concatenate([
        jnp.array([0.0]),
        jnp.cumsum(0.5 * (ds_vals[1:] + ds_vals[:-1]) * delta_s)
    ])
    s_uniform = jnp.interp(jnp.linspace(0, cumulative_length[-1], n_points_per_spline), cumulative_length, s_fine)
    return s_uniform, cumulative_length[-1]

def downsample_points(coords):
    x_spline, y_spline = fit_spline(coords)
    s_uniform, length = get_uniform_points(x_spline, y_spline, n_points_per_spline=8)
    x = x_spline(s_uniform)
    y = y_spline(s_uniform)
    return jnp.stack([x, y], axis=-1)

In [4]:
i = 2
images = []
init_splines = []
for i in [2, 3]:
    img = dat[i, 1:].reshape(28, 28)
    img = img / 255.0
    images.append(img)
    path = get_splines_from_frame(img, 0.1)[0][:, ::-1]
    print(len(path))
    init_spline = np.array(path).astype(np.float32)
    init_spline = downsample_points(init_spline)
    init_splines.append(init_spline[None])
    print(init_spline.shape)
    plt.imshow(img, cmap='gray')
    plt.plot(path[:, 0], path[:, 1], 'r-')
    plt.plot(init_spline[:, 0], init_spline[:, 1], 'g-')
    plt.show()

In [5]:
images = jnp.array(images)

In [6]:
init_splines = jnp.array(init_splines) / 28

In [7]:
model = SplenderImage(random.PRNGKey(0), init_splines, res = 28)

In [8]:
filter_spec = jtu.tree_map(lambda _: True, model)
filter_background = eqx.tree_at(
        lambda tree: (tree.loc_params, tree.knot_params, tree.global_scale, tree.brush_profile, tree.kernel, tree.opacity),
        filter_spec,
        replace=(False, False, False, False, False, False),
    )
filter_splines = eqx.tree_at(
        lambda tree: (tree.knot_params, tree.global_scale, tree.brush_profile, tree.kernel, tree.opacity),
        filter_spec,
        replace=(False, False, False, False, False),
    )
# filter_splines = eqx.tree_at(
#         lambda tree: (tree.brush_profile, tree.kernel, tree.opacity),
#         filter_spec,
#         replace=(False, False, False),
#     )

In [9]:
@eqx.filter_jit
def make_step(model, images, opt_state, filter):
    @eqx.filter_value_and_grad
    def loss(diff_model, static_model, images):
        model = eqx.combine(diff_model, static_model)
        recon, lengths, curvatures = model()
        recon_loss = jnp.mean((images - recon) ** 2)
        min_scale_knots = jax.nn.sigmoid((model.loc_params + model.knot_params)[..., 2]).min(axis = -1)
        # jax.debug.print("min_scale_knots: {min_scale_knots}", min_scale_knots=min_scale_knots)
        scale_multiplier_reg = 1e-3 * ((min_scale_knots - 1.0)**2).mean()
        return recon_loss + scale_multiplier_reg
    
    diff_model, static_model = eqx.partition(model, filter)
    loss_value, grads = loss(diff_model, static_model, images)
    # jax.debug.print("loss: {loss_value}", loss_value=loss_value)
    # jax.debug.print("grads: {grads}", grads=grads)
    updates, opt_state = optim.update(grads, opt_state)
    model = eqx.apply_updates(model, updates)
    return model, opt_state, loss_value

In [10]:
optim = optax.adam(1e-2)
losses = []
opt_state = optim.init(model)
for step in range(199):
    model, opt_state, loss_value = make_step(model, images, opt_state, filter_background)
    losses.append(loss_value)


In [ ]:
optim = optax.adam(1e-4)
opt_state = optim.init(model)
for step in range(1):
    model_s, opt_state, loss_value = make_step(model, images, opt_state, filter_spec)
    # model_s, opt_state, loss_value = make_step(model, images, opt_state, filter_splines)
    losses.append(loss_value)

In [ ]:
plt.plot(losses)
# plt.ylim(0.0665, 0.0668)

In [ ]:
ims, lengths, curvatures = model()

In [ ]:
print("model.loc_params", model.loc_params)
print("model.knot_params", model.knot_params)
print("model.global_scale", model.global_scale)
print("model.brush_profile", model.brush_profile)
print("model.kernel", model.kernel)
print("model.opacity", model.opacity)

In [ ]:
model_s.knot_params

In [ ]:
ims.shape

In [ ]:
plt.imshow(ims[0], cmap='gray', vmin=0, vmax=1)
plt.show()
plt.imshow(images[0], cmap='gray', vmin=0, vmax=1)
plt.show()
# plt.imshow(ims[1], cmap='gray', vmin=0, vmax=1)
plt.imshow(ims[1], cmap='gray')
plt.show()